In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor, AbsCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

use_relax = True
use_solid_steady = False

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFPSP%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, steady=use_solid_steady, time_order=2)
solverS.output_file = 'solid_outputs/resFPSP%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_temperature_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=inodes, value=300.0)
abTempS = InterfaceData(size=inodes, value=T_bot)
TempF = InterfaceData(size=inodes, value=300.0)
TempS = InterfaceData(size=inodes, value=T_bot)
thetaT = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = AbsCovergenceMonitor(tol=1e-4)

# maximum pc steps allowed
max_pc_steps = 200

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
hF2S = hF[f2s]
hS = solverS.get_interface_h(tag=interface_tag)
hS2F = hS[s2f]

CF1 = hS2F/(hS2F+hF)
CF2 = hF/(hS2F+hF)

CS1 = hS/(hF2S+hS)
CS2 = hF2S/(hF2S+hS)

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
Nmax = 8000
dt = 0.05
t = 0.0
avg_pc_iterations = 0.0
flog = open('FPSP%i.log' % k, mode='w')
flog.write('Fluid pseudo Robin with solid pseudo Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FPSP%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    # update fluid interface temperature
    TempF.curr[:] = CF1 * abTempS.curr[s2f] + CF2 * abTempF.curr
    
    solverF.put_temperature_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis,
        data=TempF.curr
    )
    
    while True:
        # back up previous interface value
        TempF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid ambient temperature
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # retrieve solid ambient temperature
        abTempS.curr[:] = solverS.get_interface_ambient_temperature(
            tag=interface_tag,
            h_given = hS
        )
        
        # update solid interface temperature
        TempS.curr[:] = CS1 * abTempS.curr + CS2 * abTempF.curr[f2s]
        # update solid temperature interface
        solverS.update_temperature_interface(
            tag=interface_tag,
            T=TempS.curr
        )
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid ambient temperature, the new one
        abTempS.curr[:] = solverS.get_interface_ambient_temperature(
            tag=interface_tag,
            h_given = hS
        )
        
        # update fluid interface temperature
        TempF.curr[:] = CF1 * abTempS.curr[s2f] + CF2 * abTempF.curr
        # update residual
        TempF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(TempF)
        if is_conv or pc_counts >= max_pc_steps:
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(TempF)
                under_relax.update_solution(TempF)
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=TempF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()
            pc_counts = pc_counts + 1

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        fluid_ofile.write(solverF, t)
        solverS.write()
    thetaT.backup()
    thetaT.curr[:] = (solverF.get_temperature_interface(3,0,1)-300)/10
    thetaT.update_res()
    is_conv = conv_solution.determine_convergence(thetaT)
    if is_conv:
        if step % 50 and step != Nmax-1:
            solverS.write()
            fluid_ofile.write(solverF, t)
        break
    
flog.close()

step=0, pc_iterations=8.
step=1, pc_iterations=3.
step=2, pc_iterations=3.
step=3, pc_iterations=3.
step=4, pc_iterations=3.
step=5, pc_iterations=2.
step=6, pc_iterations=2.
step=7, pc_iterations=2.
step=8, pc_iterations=2.
step=9, pc_iterations=2.
step=10, pc_iterations=2.
step=11, pc_iterations=2.
step=12, pc_iterations=2.
step=13, pc_iterations=2.
step=14, pc_iterations=2.
step=15, pc_iterations=2.
step=16, pc_iterations=2.
step=17, pc_iterations=2.
step=18, pc_iterations=2.
step=19, pc_iterations=2.
step=20, pc_iterations=2.
step=21, pc_iterations=2.
step=22, pc_iterations=2.
step=23, pc_iterations=2.
step=24, pc_iterations=2.
step=25, pc_iterations=2.
step=26, pc_iterations=2.
step=27, pc_iterations=2.
step=28, pc_iterations=2.
step=29, pc_iterations=2.
step=30, pc_iterations=2.
step=31, pc_iterations=2.
step=32, pc_iterations=2.
step=33, pc_iterations=2.
step=34, pc_iterations=2.
step=35, pc_iterations=2.
step=36, pc_iterations=2.
step=37, pc_iterations=2.
step=38, pc_iterations

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat(
    'numeric_res/kFPSP%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)

[0.9265268  0.93162898 0.93621803 0.93888361 0.94120159 0.94335248
 0.94526674 0.94701686 0.94871651 0.95024019 0.95167375 0.95301088
 0.95431437 0.95553069 0.95669644 0.9577866  0.95885283 0.95986105
 0.96081736 0.96174939 0.96263086 0.96348447 0.96430295 0.96508093
 0.96584024 0.96656655 0.96727194 0.96793609 0.96857011 0.96918934
 0.9697875  0.97035753 0.97091183 0.97144702 0.9719599  0.97244962
 0.97292938 0.97338895 0.97383677 0.97425997 0.97466589 0.97505658
 0.97543658 0.9758028  0.97615555 0.97649396 0.97682252 0.97713421
 0.97743356 0.97772769 0.9780075  0.97827388 0.97853018 0.9787773
 0.97901019 0.97923646 0.97944908 0.97965366 0.97984972 0.9800327
 0.98020519 0.98037307 0.980529   0.98067426 0.98080862 0.98093726
 0.98105122 0.98115836 0.98125179 0.98133667 0.98141584 0.9814799
 0.98153142 0.98157754 0.98160959 0.98163048 0.98163745 0.981635
 0.98161557 0.98158627 0.981539   0.98147869 0.98140427 0.98131469
 0.98120548 0.98107421 0.98093043 0.98075938 0.98056176 0.98034317
